In [71]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.colheader_justify', 'left') 
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.width', None)  

In [72]:
#file_path = '/home/carolus/Documents/school/green_ia/data/00_data/00_train_02.jsonl'
file_path = '/home/carolus/Documents/school/green_ia/scripts/test_model/validation_00.jsonl'
num_samples = 500

In [73]:
#def count_objects_in_jsonl(file_path):
#    count = 0
#    with open(file_path, 'r', encoding='utf-8') as file:
#        for line in file:
#            count += 1
#    return count
#print(f"nombre objets {file_path}: {count_objects_in_jsonl(file_path)}")


In [74]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df
df = sample_jsonl_file(file_path, num_samples)

In [75]:
#occurrences = df['food_group'].value_counts()
#print(occurrences)

In [76]:
#df.tail(1)

In [77]:
df.head(100)

pnns_1  \
0   4.0      
1   9.0      
2   3.0      
3   4.0      
4   5.0      
5   3.0      
6   1.0      
7   5.0      
8   7.0      
9   5.0      
10  3.0      
11  3.0      
12  0.0      
13  7.0      
14  4.0      
15  1.0      
16  0.0      
17  1.0      
18  0.0      
19  7.0      
20  7.0      
21  4.0      
22  1.0      
23  0.0      
24  4.0      
25  0.0      
26  4.0      
27  3.0      
28  3.0      
29  4.0      
30  4.0      
31  1.0      
32  9.0      
33  5.0      
34  1.0      
35  4.0      
36  4.0      
37  9.0      
38  1.0      
39  4.0      
40  3.0      
41  9.0      
42  5.0      
43  9.0      
44  5.0      
45  4.0      
46  4.0      
47  0.0      
48  0.0      
49  3.0      
50  4.0      
51  4.0      
52  5.0      
53  4.0      
54  5.0      
55  4.0      
56  4.0      
57  0.0      
58  3.0      
59  2.0      
60  0.0      
61  0.0      
62  0.0      
63  4.0      
64  6.0      
65  4.0      
66  1.0      
67  1.0      
68  5.0      
69  7.0      
70  4.0      
71  3.0      
72  0.0      
73  4.0      
74  2.0      
75  0.0      
76  7.0      
77  9.0      
78  7.0      
79  1.0      
80  4.0      
81  4.0      
82  1.0      
83  1.0      
84  4.0      
85  4.0      
86  6.0      
87  4.0      
88  0.0      
89  6.0      
90  0.0      
91  4.0      
92  0.0      
93  7.0      
94  1.0      
95  7.0      
96  1.0      
97  0.0      
98  4.0      
99  0.0      

   name                                                                      \
0                                                              skyr kinuski   
1                                                      organic green olives   
2                                        pomme de nouvelle-aquitaine fraise   
3                                                   valiojogurtti - banaani   
4                                                               baked beans   
5                                                             rote zwiebeln   
6                                  tomato ketchup 50% weniger zucker & salz   
7                                                 lentilles vertes du berry   
8                                          black ice tea lemon & lemongrass   
9                                                     farine de blé t65 bio   
10                                  compote d’abricots rouges du roussillon   
11                         poêlée de légumes de normandie cuisinée au cidre   
12                                                      griottes à tartiner   
13                                                          sirop grenadine   
14                                                          te descafeinado   
15                                                   pasta sauce arrabbiata   
16                                                            miel d'acacia   
17                                  bechamel de arroz ecológica sin lactosa   
18                                                        miel d'eucalyptus   
19                                                    sirop de menthe verte   
20                            maté fruité bio - 100 g - les jardins de gaïa   
21                                                  herbal infusion rosehip   
22                                                   sauce pizza à l’origan   
23                                          petit beurre nature sans sucres   
24                                                    chicorée café soluble   
25                                                                   sukker   
26                                                       1919 saveur amande   
27                   frullato 100 frutta mela yogurt banana mirtilli miglio   
28                                                                   pommes   
29                                                           pfefferminztee   
30                                                                     dill   
31                                                           

In [78]:
df['ecart'] = abs(df['ecoscore_tags'] - df['predicted_ecoscore_tags'])
pourcentage_ecarts_superieurs = (df['ecart'] > 0.49).mean() * 100
print(f"Pourcentage de valeurs avec un écart supérieur à 0.49 : {pourcentage_ecarts_superieurs:.2f}%")

Pourcentage de valeurs avec un écart supérieur à 0.49 : 14.86%


In [79]:
#col_nan = 'additives'
#nb_empty = (df[col_nan].isna()).sum() 
##nb_empty = (df[col_nan] == 'empty').sum() 
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in {col_nan}: {pourcentage_empty:.2f}%")

In [80]:
def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"Warning: column '{column_name}' does not exist.")
        return

    # Filtrer les valeurs NaN et 'empty'
    filtered_df = df[df[column_name].notna() & (df[column_name] != 'empty')]
    
    # Compter les occurrences des valeurs restantes
    data = filtered_df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    
    # Créer le treemap
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',
    ))
    
    # Mettre à jour la mise en page du treemap
    fig.update_layout(
        title=f'Treemap for column: {column_name}',
        paper_bgcolor='white',
        plot_bgcolor='white',
        font_color='black'
    )
    fig.show()

In [81]:
colonnes_a_travailler = ['stores', 'ecoscore_tags', 'ecoscore_score', 'countries', 'food_group', 'nova', 'palm_oil', 'nutrient_level', 'categories', 'nutriscore_tags', 'additives', 'main_category', 'pnns_1']  
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [82]:
#nb_empty = (df['ecoscore_score'] == 44).sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in ecoscore_score before NaN reduction : {pourcentage_empty:.2f}%")

In [83]:
nb_empty = (df['stores'] == 'empty').sum()
nb_total = len(df)
pourcentage_empty = (nb_empty / nb_total) * 100
print(f"percetange empty in countries before NaN reduction : {pourcentage_empty:.2f}%")

percetange empty in countries before NaN reduction : 63.43%


In [84]:
#nb_empty = df['ecoscore_tags'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in categories before NaN reduction : {pourcentage_empty:.2f}%")

In [85]:
#file_path_csv = '/home/carolus/Documents/school/green_ia/scripts/validation_predictions.csv'
#df_csv = pd.read_csv(file_path_csv)

In [86]:
#df_csv.head(60)

In [87]:
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#max_diff = df_csv['difference'].max()
#min_diff = df_csv['difference'].min()
#mean_diff = df_csv['difference'].mean()  
#median_diff = df_csv['difference'].median()  
#total_rows = df_csv.shape[0]
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#count_greater_than_2 = (df_csv['difference'].abs() > 2).sum()
#percentage = (count_greater_than_2 / total_rows) * 100

#print(f"Écart maximum: {max_diff}")
#print(f"Écart minimum: {min_diff}")
#print(f"Écart moyen: {mean_diff}")
#print(f"Écart médian: {median_diff}")
#print(f"poucentage lignes écart supérieur à 2% : {percentage}%")